this is some markdown


In [2]:
import numpy as np
from os import listdir
from os.path import isfile, join
import h5py
from PIL import Image

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.optimizers import SGD

from sklearn.manifold import TSNE


5


In [ ]:
def get_image(path, w, h):
    img = Image.open(path)
    if img.mode == 'L' or img.mode == 'RGBA':
        img = img.convert('RGB')
    img = img.resize((w, h), Image.ANTIALIAS)
    im2 = np.array(img.getdata(), np.uint8)
    im3 = np.array(img.getdata(), np.uint8)
    im2[:,0] = im3[:,2]
    im2[:,2] = im3[:,0]
    im2 = im2.reshape(img.size[1], img.size[0], 3).astype(np.float32)
    im2[:,:,0] -= 103.939
    im2[:,:,1] -= 116.779
    im2[:,:,2] -= 123.68
    im2 = im2.transpose((2,0,1))
    im2 = np.expand_dims(im2, axis=0)
    return im2


In [ ]:
def VGG_16(weights_path):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    f = h5py.File(weights_path)
    for k in range(f.attrs['nb_layers']):
        if k >= len(model.layers):
            break
        g = f['layer_{}'.format(k)]
        weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
        model.layers[k].set_weights(weights)
    return model

In [ ]:

img_width = 224 # use this for vgg16
img_height = 224
vgg_path = '/Users/gene/Learn/keras-rtst/vgg16_weights.h5'
images_path = '/Users/gene/Code/of_v0.9.0_osx_release/addons/ofxTSNE_2/example-images/bin/data/animals'
tsne_path = 'cnn_tsne_vis/data/tsne_points.txt'
skip = 5 	# use fewer images by skipping through directory more (1 = use all the images)



# load model
model = VGG_16(vgg_path)


In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

# get images
images = [f for f in listdir(images_path) if isfile(join(images_path, f))]
images = [images[i] for i in range(1,len(images), skip)]

# analyze images and grab activations
activations = []
for idx,image_path in enumerate(images):
    print "getting activations for %s %d/%d" % (image_path,idx,len(images))
    file_path = join(images_path,image_path)
    image = get_image(file_path, img_width, img_height)
    acts = model.predict(image)[0]
    activations.append(acts)


In [ ]:
X = np.array(activations)
tsne = TSNE(n_components=2, perplexity=30, verbose=2).fit_transform(X)


# normalize t-sne points to {0,1}
x, y = tsne[:,0], tsne[:,1]
x = (x-np.min(x)) / (np.max(x) - np.min(x))
y = (y-np.min(y)) / (np.max(y) - np.min(y))


In [ ]:
f = open(tsne_path, 'w')
for idx,image_path in enumerate(images):
    file_path = join(images_path,image_path)
    line = "%s, %f, %f" % (file_path, x[idx], y[idx])
    f.write('%s\n'%line)

f.close()
print "done! results saved to %s"%tsne_path